### Импортируем необходимые библиотеки для работы с данными 

In [1]:
import pandahouse as ph
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import hashlib # Делать хеши
import swifter # Ускорялка метода apply
from scipy import stats # Статистика
from scipy.stats import norm, ttest_ind
from tqdm import tqdm
%matplotlib inline

In [2]:
#параметры соединения - нужны, чтобы подключиться к нужной схеме данных
connection = {'host': 'https://clickhouse.lab.karpov.courses',
'database':'simulator_20250220',
'user':'student',
'password':'dpo_python_2020'
}

#текст запроса
query = """
select views, count() as users
from (select  user_id,
              sum(action = 'view') as views
from  {db}.feed_actions
where toDate(time) between '2025-01-24' and '2025-01-30'
group by user_id
)
group by views
order by views
"""

# эта функция выполнит запрос и запишет его результат в pandas DataFrame
views_distribution = ph.read_clickhouse(query, connection=connection)

In [3]:
views_distribution.head()

,views,users
0,1,4
1,2,1
2,3,4
3,4,5
4,5,18


In [4]:
# Количество юзеров в каждую группу для симуляций
sample_size = views_distribution.users.sum() // 2
sample_size

20998

### Нормализуем данные и добавим колонку с вероятностью появления разного количества просмотров

In [5]:
views_distribution['p'] = views_distribution['users']/views_distribution.users.sum()

In [6]:
views_distribution.sort_values(by = 'p', ascending = False)

,views,users,p
15,16,545,0.012977
14,15,537,0.012787
13,14,500,0.011906
34,35,485,0.011548
29,30,469,0.011167
...,...,...,...
280,287,1,0.000024
278,285,1,0.000024
276,280,1,0.000024
1,2,1,0.000024


In [7]:
q = """
select 
   floor(ctr, 2) as ctr, count() as users
from (select toDate(time) as dt,
            user_id,
            sum(action = 'like')/sum(action = 'view') as ctr
from {db}.feed_actions
where dt between '2025-01-24' and '2025-01-30'
group by dt, user_id
)
group by ctr
"""


ctr_distribution = ph.read_clickhouse(q, connection=connection)
ctr_distribution['p'] = ctr_distribution['users']/ctr_distribution.users.sum()

### Добавим колонку с вероятностью появления разного CTR

In [8]:
ctr_distribution.sort_values(by = 'p', ascending = False)

,ctr,users,p
17,0.20,4993,0.058658
33,0.16,4233,0.049729
50,0.25,4216,0.049529
6,0.18,4213,0.049494
72,0.21,3957,0.046487
...,...,...,...
12,0.81,2,0.000023
73,0.83,1,0.000012
20,0.73,1,0.000012
51,1.00,1,0.000012


### Зададим распределения просмотров и CTR и проведём симуляции A/B - тестов

In [9]:
rng = np.random.default_rng()

In [10]:
p_values = []
for _ in tqdm(range(20000)):
    group_A_views = rng.choice(views_distribution['views'], size=(sample_size), replace=True, p=views_distribution['p']).astype(np.int64)
    group_B_views = rng.choice(views_distribution['views'], size=(sample_size), replace=True, p=views_distribution['p']).astype(np.int64)
    group_B_views = group_B_views + ((1 + rng.binomial(n=1, p=0.5, size=sample_size)) * rng.binomial(n=1, p=0.9, size=sample_size) * (group_B_views >= 50))

    group_A_ctr = rng.choice(ctr_distribution['ctr'], size=(sample_size), replace=True, p=ctr_distribution['p'])
    group_B_ctr = rng.choice(ctr_distribution['ctr'], size=(sample_size), replace=True, p=ctr_distribution['p'])
    
    likes_A = rng.binomial(group_A_views, group_A_ctr)
    likes_B = rng.binomial(group_B_views, group_B_ctr)
    
    #mask_A = group_A_views >= 30
    #mask_B = group_B_views >= 30
    
    p_values.append(stats.ttest_ind(likes_A, likes_B, equal_var=False).pvalue)

100%|██████████| 20000/20000 [04:14<00:00, 78.59it/s]


### Расчитаем мощность теста

In [11]:
power = len([i for i in p_values if i < 0.05]) * 100 / len(p_values)
power

26.355